Transaction validation

* Inputs are previously unspent
* Sum of inputs is greater than or equal to the outputs
* The SigScript unlocks the previous ScriptPubKey


In [2]:
# Example - is this transaction attempting to make money out of thin air?

from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf830\
3c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccf\
cf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8\
e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278\
afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88a\
c99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
print(raw_tx)
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)  # <1>

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600
True


ECDSA requires the public key P, the signature hash z, and the signature (r,s)

In [3]:
from ecc import S256Point, Signature
# SEC - standards for efficent cryptography - public key serialization
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e\
213bf016b278a')
# DER - distinguised encoding rules - for signatures
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031c\
cfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9\
c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


Getting the signature hash

* Can't hash the transaction, since the signature itself is embedded in the SigScript
* Transaction is modified before signing - a signature hash for each input is computed

The process:

1. Empty all ScriptSigs before checking (or for creating, probably has empty ScriptSigs when creating).
2. Replace the SigScript of the transaction with ScriptPubKey of the previous transaction output
3. Append the hash type encoded little-endian over 4 bytes

The hash256 of the modified transaction can be taken, and the big-endian integer is taken as z

In [5]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914\
bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896\
bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02\
e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288a\
c1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))


0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [6]:
# Putting it all together

from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e\
213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031c\
cfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9\
c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

In [1]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b"test source"))
private_key = PrivateKey(secret)
private_key.point.address(compressed=True, testnet=True)

'mzTWLA2kJwZECywvGmkj8k2tVmVDpXLNq6'

In [2]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret2 = little_endian_to_int(hash256(b"test destination"))
private_key2 = PrivateKey(secret2)
private_key2.point.address(compressed=True, testnet=True)

'mivh1dcR1Puy9NLCVzRnsoyBEqRrkmP3a4'

In [5]:
# Fund the address
!curl -X POST --data '{"address": "mzTWLA2kJwZECywvGmkj8k2tVmVDpXLNq6", "amount": 0.02}' http://localhost:3000/faucet

{"txId":"77aed9d22ac34c4ec4554f48ef16d4b6f20a3f04806eee70972d7eb8e5802fc5"}



Can we send from the address controlled by our private key to the second address, receiving change back to our original address (not a recommended practive, but for simplicity)?

In [6]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Source address has 2,000,000 satoshis - send half, guess 2000 for the txn fee, rest is change back 998,000

prev_tx = bytes.fromhex('77aed9d22ac34c4ec4554f48ef16d4b6f20a3f04806eee70972d7eb8e5802fc5')
prev_index = 0
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(998000)  
change_h160 = decode_base58('mzTWLA2kJwZECywvGmkj8k2tVmVDpXLNq6')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(1000000)  
target_h160 = decode_base58('mivh1dcR1Puy9NLCVzRnsoyBEqRrkmP3a4')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)  
print(tx_obj)
print(tx_obj.serialize().hex())



tx: 7e9f1fa424900f3b493b910fe149d6de6ad9ec647723cb8dab732a2789ff2160
version: 1
tx_ins:
77aed9d22ac34c4ec4554f48ef16d4b6f20a3f04806eee70972d7eb8e5802fc5:0
tx_outs:
998000:OP_DUP OP_HASH160 cfc377f37ec1c62f778b4518abda888c4303b71a OP_EQUALVERIFY OP_CHECKSIG
1000000:OP_DUP OP_HASH160 25659a4b6caaf192c0ad254a929208f7b6069630 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
0100000001c52f80e5b87e2d9770ee6e80043f0af2b6d416ef484f55c44e4cc32ad2d9ae770000000000ffffffff02703a0f00000000001976a914cfc377f37ec1c62f778b4518abda888c4303b71a88ac40420f00000000001976a91425659a4b6caaf192c0ad254a929208f7b606963088ac00000000


What does this look like if we decode it using the command line?

```
export RAW_TX=0100000001188e4a1df9fe602e8d627c2492374a3b8d79f496e7412d1d5044326d057bd18f0100000000ffffffff02703a0f00000000001976a91428145ab8d99d9d9d3197693fad5e389f03cf1c9488ac40420f00000000001976a9144c73ec04ab2c8658acb9017dcb0c18a7d7eb909488ac00000000

bcli decoderawtransaction $RAW_TX
{
  "txid": "47775e4ce1cb4550875bfba71b40ae6c2379ce18e900166226276fb647790682",
  "hash": "47775e4ce1cb4550875bfba71b40ae6c2379ce18e900166226276fb647790682",
  "version": 1,
  "size": 119,
  "vsize": 119,
  "weight": 476,
  "locktime": 0,
  "vin": [
    {
      "txid": "8fd17b056d3244501d2d41e796f4798d3b4a3792247c628d2e60fef91d4a8e18",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 0.00998000,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 28145ab8d99d9d9d3197693fad5e389f03cf1c94 OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mjAshyUhcMxyP34vdeED6JByENT5SNZqaA)#x0ucj78a",
        "hex": "76a91428145ab8d99d9d9d3197693fad5e389f03cf1c9488ac",
        "address": "mjAshyUhcMxyP34vdeED6JByENT5SNZqaA",
        "type": "pubkeyhash"
      }
    },
    {
      "value": 0.01000000,
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 4c73ec04ab2c8658acb9017dcb0c18a7d7eb9094 OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mnVCX2m5UEapYf9UrvJj17D9i72XqikvHW)#m3f8ue7w",
        "hex": "76a9144c73ec04ab2c8658acb9017dcb0c18a7d7eb909488ac",
        "address": "mnVCX2m5UEapYf9UrvJj17D9i72XqikvHW",
        "type": "pubkeyhash"
      }
    }
  ]
}
```

In [7]:
# Sign the transaction

from ecc import PrivateKey
from helper import SIGHASH_ALL

print(tx_obj.sign_input(0, private_key)) 
print(tx_obj.serialize().hex())

3045022100ca8bc63b33b03c0c4083b7fd54da78a83cd47adb77b6dce35a0ab8821dc762cf02204d092271888c0df8880d58c7f9ed4b76500c1b2b0ac7a5f2eb9344684c143fad01 0236248429d51fdbe88896767819e5d573ab3f76dd7228da765e43da9937b8fa8d OP_DUP OP_HASH160 cfc377f37ec1c62f778b4518abda888c4303b71a OP_EQUALVERIFY OP_CHECKSIG
True
0100000001c52f80e5b87e2d9770ee6e80043f0af2b6d416ef484f55c44e4cc32ad2d9ae77000000006b483045022100ca8bc63b33b03c0c4083b7fd54da78a83cd47adb77b6dce35a0ab8821dc762cf02204d092271888c0df8880d58c7f9ed4b76500c1b2b0ac7a5f2eb9344684c143fad01210236248429d51fdbe88896767819e5d573ab3f76dd7228da765e43da9937b8fa8dffffffff02703a0f00000000001976a914cfc377f37ec1c62f778b4518abda888c4303b71a88ac40420f00000000001976a91425659a4b6caaf192c0ad254a929208f7b606963088ac00000000


Send the transaction

```
export SIGNED=0100000001c52f80e5b87e2d9770ee6e80043f0af2b6d416ef484f55c44e4cc32ad2d9ae77000000006b483045022100ca8bc63b33b03c0c4083b7fd54da78a83cd47adb77b6dce35a0ab8821dc762cf02204d092271888c0df8880d58c7f9ed4b76500c1b2b0ac7a5f2eb9344684c143fad01210236248429d51fdbe88896767819e5d573ab3f76dd7228da765e43da9937b8fa8dffffffff02703a0f00000000001976a914cfc377f37ec1c62f778b4518abda888c4303b71a88ac40420f00000000001976a91425659a4b6caaf192c0ad254a929208f7b606963088ac00000000



bcli -regtest sendrawtransaction $SIGNED
6f536844530e97a11bfe8db1db88bd766b6004a9f99805ec94ac333d2b877f28

curl localhost:3000/tx/6f536844530e97a11bfe8db1db88bd766b6004a9f99805ec94ac333d2b877f28 | jq
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1699  100  1699    0     0   129k      0 --:--:-- --:--:-- --:--:--  237k
{
  "txid": "6f536844530e97a11bfe8db1db88bd766b6004a9f99805ec94ac333d2b877f28",
  "version": 1,
  "locktime": 0,
  "vin": [
    {
      "txid": "77aed9d22ac34c4ec4554f48ef16d4b6f20a3f04806eee70972d7eb8e5802fc5",
      "vout": 0,
      "prevout": {
        "scriptpubkey": "76a914cfc377f37ec1c62f778b4518abda888c4303b71a88ac",
        "scriptpubkey_asm": "OP_DUP OP_HASH160 OP_PUSHBYTES_20 cfc377f37ec1c62f778b4518abda888c4303b71a OP_EQUALVERIFY OP_CHECKSIG",
        "scriptpubkey_type": "p2pkh",
        "scriptpubkey_address": "mzTWLA2kJwZECywvGmkj8k2tVmVDpXLNq6",
        "value": 2000000
      },
      "scriptsig": "483045022100ca8bc63b33b03c0c4083b7fd54da78a83cd47adb77b6dce35a0ab8821dc762cf02204d092271888c0df8880d58c7f9ed4b76500c1b2b0ac7a5f2eb9344684c143fad01210236248429d51fdbe88896767819e5d573ab3f76dd7228da765e43da9937b8fa8d",
      "scriptsig_asm": "OP_PUSHBYTES_72 3045022100ca8bc63b33b03c0c4083b7fd54da78a83cd47adb77b6dce35a0ab8821dc762cf02204d092271888c0df8880d58c7f9ed4b76500c1b2b0ac7a5f2eb9344684c143fad01 OP_PUSHBYTES_33 0236248429d51fdbe88896767819e5d573ab3f76dd7228da765e43da9937b8fa8d",
      "is_coinbase": false,
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "scriptpubkey": "76a914cfc377f37ec1c62f778b4518abda888c4303b71a88ac",
      "scriptpubkey_asm": "OP_DUP OP_HASH160 OP_PUSHBYTES_20 cfc377f37ec1c62f778b4518abda888c4303b71a OP_EQUALVERIFY OP_CHECKSIG",
      "scriptpubkey_type": "p2pkh",
      "scriptpubkey_address": "mzTWLA2kJwZECywvGmkj8k2tVmVDpXLNq6",
      "value": 998000
    },
    {
      "scriptpubkey": "76a91425659a4b6caaf192c0ad254a929208f7b606963088ac",
      "scriptpubkey_asm": "OP_DUP OP_HASH160 OP_PUSHBYTES_20 25659a4b6caaf192c0ad254a929208f7b6069630 OP_EQUALVERIFY OP_CHECKSIG",
      "scriptpubkey_type": "p2pkh",
      "scriptpubkey_address": "mivh1dcR1Puy9NLCVzRnsoyBEqRrkmP3a4",
      "value": 1000000
    }
  ],
  "size": 226,
  "weight": 904,
  "fee": 2000,
  "status": {
    "confirmed": false
  }
}
```

Note: this example only works with compressed addresses

In [8]:
## This is an example that does not have a change address - sends everything to a target address minus a small fee


from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Source address has 2,000,000 satoshis - send half, guess 2000 for the txn fee, rest is change back 998,000

prev_tx = bytes.fromhex('448ea0ff9dee6f967bebca84463d017d246ce081b221313c58ccf06c76b18ac3')
prev_index = 1
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []

target_amount = int(2000000-2000)  
target_h160 = decode_base58('mivh1dcR1Puy9NLCVzRnsoyBEqRrkmP3a4')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)

tx_obj = Tx(1, [tx_in], [target_output], 0, True)  
print(tx_obj)
print(tx_obj.serialize().hex())

tx: 4cd3e5363e74ed9e934a2ae5f1c1a8f19e73f3b67539efbe104dca3d10595b05
version: 1
tx_ins:
448ea0ff9dee6f967bebca84463d017d246ce081b221313c58ccf06c76b18ac3:1
tx_outs:
1998000:OP_DUP OP_HASH160 25659a4b6caaf192c0ad254a929208f7b6069630 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
0100000001c38ab1766cf0cc583c3121b281e06c247d013d4684caeb7b966fee9dffa08e440100000000ffffffff01b07c1e00000000001976a91425659a4b6caaf192c0ad254a929208f7b606963088ac00000000


In [4]:
print(tx_obj.sign_input(0, private_key)) 
print(tx_obj)
print(tx_obj.serialize().hex())

3044022018333f4ec707973dd7c905f0dbfc6ac5401cad98dd0279595559bac9d9571b120220303df21061a311dd4e50bd4aa2c1452a4e1544c700fa1d13d03ca8bebffe431301 0236248429d51fdbe88896767819e5d573ab3f76dd7228da765e43da9937b8fa8d OP_DUP OP_HASH160 cfc377f37ec1c62f778b4518abda888c4303b71a OP_EQUALVERIFY OP_CHECKSIG
True
tx: e5dde6e9b914b1c7d933cf19d4d85385a9298f91ae10e2af5c881eb85b75064d
version: 1
tx_ins:
448ea0ff9dee6f967bebca84463d017d246ce081b221313c58ccf06c76b18ac3:1
tx_outs:
1998000:OP_DUP OP_HASH160 25659a4b6caaf192c0ad254a929208f7b6069630 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
0100000001c38ab1766cf0cc583c3121b281e06c247d013d4684caeb7b966fee9dffa08e44010000006a473044022018333f4ec707973dd7c905f0dbfc6ac5401cad98dd0279595559bac9d9571b120220303df21061a311dd4e50bd4aa2c1452a4e1544c700fa1d13d03ca8bebffe431301210236248429d51fdbe88896767819e5d573ab3f76dd7228da765e43da9937b8fa8dffffffff01b07c1e00000000001976a91425659a4b6caaf192c0ad254a929208f7b606963088ac00000000
